#Importing Libraries and Dataset


In [ ]:
!pip3 install ktrain

In [2]:
#Importing libraries

import pandas as pd
import numpy as np

import ktrain
from ktrain import text
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from google.colab import files


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
!pip install -q kaggle
uploaded = files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
#Importing DataSet

!kaggle datasets download -d tunguz/200000-jeopardy-questions

 78% 9.00M/11.5M [00:00<00:00, 15.7MB/s]
100% 11.5M/11.5M [00:00<00:00, 18.1MB/s]


In [26]:
#Loading dataset

df = pd.read_csv("/content/200000-jeopardy-questions.zip")

In [64]:
ppp =pd.read_csv("/content/200000-jeopardy-questions.zip")
ppp.shape

(216930, 7)

#Preprocessing

In [27]:
#Removing None values from dataset

df =df[df[' Value'] != 'None']

In [28]:
# Considering only 10000 data items form Complete dataset

df = df.head(10000)

In [30]:

# Converting values in [ ' Value' ] column in to intgers after preprocessing each value.

df[' Value']  = df[' Value'].apply(
    lambda value: (int(value.replace(',', '').replace('$' , '')))
)


In [31]:
df.tail()

,Show Number,Air Date,Round,Category,Value,Question,Answer
10164,5825,2010-01-01,Double Jeopardy!,BEYOND .COM,2000,"As of 2010, you can invent your own domain, ac...",.org (.net accepted)
10165,5825,2010-01-01,Double Jeopardy!,PICK A PLANET,2000,Leda is its 13th moon,Jupiter
10166,5825,2010-01-01,Double Jeopardy!,THE ASSASSINATION OF LINCOLN,2000,Attempts on the lives of VP Andrew Johnson & t...,Seward
10167,5825,2010-01-01,Double Jeopardy!,B.C. & AFTER,2000,Homo erectus archaeological find on Jakarta's ...,Java Man Ray
10169,5416,2008-03-10,Jeopardy!,THE WORLD OF MR. BURNS,200,"<a href=""http://www.j-archive.com/media/2008-0...",business


In [32]:
# Creating categories by using binnig concept

# Below funciton will round the value to its nearest hundread or thousand based on the if else condition if falls below

def convert_to_categories(x):
    if int(x) < 1000:
        return np.round(int(x), -2)
    elif int(x) < 10000:
        return np.round(int(x), -3)
    else:
        return np.round(int(x), -4)

df['New_value'] = df[' Value'].apply(convert_to_categories)

In [33]:
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer,New_value
0,4680,2004-12-31,Jeopardy!,HISTORY,200,"For the last 8 years of his life, Galileo was ...",Copernicus,200
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,200
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,200,The city of Yuma in this state has a record av...,Arizona,200
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,200
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,200


In [34]:
# 2D arrays storing ["Values"]  and  ["Question"] columns series from dataset 

X_new= df.iloc[:,  5].values
y_new= df.iloc[: ,-1].values

In [35]:
print(X_new)

["For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory"
 'No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves'
 'The city of Yuma in this state has a record average of 4,055 hours of sunshine each year'
 ...
 'Attempts on the lives of VP Andrew Johnson & this Secretary of State were also part of the plot'
 "Homo erectus archaeological find on Jakarta's island by an expatriate U.S. photographer"
 '<a href="http://www.j-archive.com/media/2008-03-10_J_01.mp3">I\'ve said it before: "Family. Religion. Friendship. These are 3 demons you must slay if you wish to succeed in" this</a>']


#Creating Bag of words model

In [36]:
# Building bag of words

corpus = []

length = df.shape[0]

for i in range(len(X_new)):
  review = re.sub('[^a-zA-Z]', ' ', X_new[i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [37]:
print(len(corpus))

10000


#Split dataset into  Train and Test Data

In [38]:
# Creating train and test data split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus, y_new, test_size = 0.25, random_state = 0)

In [40]:

#Finding all unique classes ( outputs ) in dataset

classes = df['New_value'].unique()

print(type(classes[0]))

<class 'numpy.int64'>


In [41]:
# Encoding function for outputs

# It assigns a integer value for every unique class.

label =0
encode = {}

for i in classes:
  if i not in encode:
    encode[i]=label
    label+=1

In [42]:
print(encode.keys())
print( encode.values())

dict_keys([200, 400, 600, 800, 2000, 1000, 3000, 5000, 100, 300, 500, 4000, 7000, 700, 8000, 6000, 10000, 900])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])


In [44]:
#Encoding train and test datasets 

y_train = [encode[x] for x in y_train]
y_test = [encode[x] for x in y_test]

#Creating a BERT model 

In [46]:

(x_train_new,  y_train_new), (x_test_new, y_test_new), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=classes,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=200)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [47]:
#Creating a model 

model = text.text_classifier('bert', train_data=(x_train_new, y_train_new), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train_new, y_train_new), 
                             val_data=(x_test_new, y_test_new),
                             batch_size=6)


Is Multi-Label? False
maxlen is 350
done.


In [50]:
#Training the model.... Model accuracy is about 19% when it it trained on (1/21)th part of complete Dataset. 
#So on runnning the code on complete Dataset there is a high chance for getting good predications and accuracy results
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
1250/1250 [==============================] - 1581s 1s/step - loss: 2.3813 - accuracy: 0.1624 - val_loss: 2.0909 - val_accuracy: 0.1932


In [ ]:
print(len(y_test))

In [ ]:
# Predicting the metrics for the model.

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report
accuracy_score = accuracy_score(y_test, y_pred)
y_pred = model.predict(X_test)
print("Acuracy of model" , accuracy_score)
print(classification_report(y_test, y_pred))

In [53]:
predictor.save("models/bert_model")